In [61]:
# The only 3 lines you need to install and use AMPL with any solver on Colab
%pip install -q amplpy
from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=["coin"],  # modules to install
    license_uuid="default",  # license to use
)  # instantiate AMPL object and register magics

Using default Community Edition License for Colab. Get yours at: https://ampl.com/ce
Licensed to AMPL Community Edition License for the AMPL Model Colaboratory (https://ampl.com/colab).


In [62]:
%%writefile evacuacao.mod
# Modelo de evacuação em AMPL

# Parâmetros
set Veiculos;  # Conjunto de veículos (1 a 6)
set Familias;  # Conjunto de tipos de famílias (1 a 5)

param Custo{Veiculos};          # Custo por viagem de cada veículo
param Capacidade_Pessoas{Veiculos};  # Capacidade de transporte de pessoas por veículo
param Capacidade_Bagagem{Veiculos};  # Capacidade de transporte de bagagem (toneladas)
param Tempo_Viagem{Veiculos};   # Tempo de viagem por veículo (em minutos)
param Max_Viagens{Veiculos};    # Número máximo de viagens por veículo
param Bagagem_Total;            # Bagagem total a transportar (em toneladas)
param Pessoas_Total;            # Pessoas totais a transportar
param Tempo_Max;                # Tempo máximo disponível para viagens
param Tamanho_Familia{Familias};  # Tamanho das famílias por tipo
param Quantidade_Familias{Familias};  # Quantidade de famílias por tipo

# Parâmetros de total de pessoas
param Homens_Total;  # Total de homens
param Mulheres_Total;  # Total de mulheres
param Criancas_Total;  # Total de crianças

# Variáveis de decisão
var x{Veiculos} >= 0, integer;  # Número de viagens por veículo
var y{Familias, Veiculos} >= 0, integer;  # Famílias do tipo i no veículo j
var h{Veiculos} >= 0, integer;  # Número de homens transportados por veículo

# Função objetivo: minimizar o custo total das viagens
minimize Custo_Total:
    sum {v in Veiculos} Custo[v] * x[v];

# Restrições
# Transporte de pessoas (famílias)
subject to Restricao_Pessoas:
    sum {v in Veiculos} Capacidade_Pessoas[v] * x[v] <= Pessoas_Total;

# Transporte de bagagem
subject to Restricao_Bagagem:
    sum {v in Veiculos} Capacidade_Bagagem[v] * x[v] <= Bagagem_Total;

# Tempo total disponível para viagens
subject to Restricao_Tempo:
    sum {v in Veiculos} Tempo_Viagem[v] * x[v] <= Tempo_Max;

# Famílias alocadas por veículo respeitando a capacidade
subject to Capacidade_Familias {v in Veiculos}:
    sum {f in Familias} Tamanho_Familia[f] * y[f, v] <= Capacidade_Pessoas[v] * x[v];

# Todas as famílias devem ser transportadas
subject to Transporte_Familias {f in Familias}:
    sum {v in Veiculos} y[f, v] = Quantidade_Familias[f];

# Restrições para alocar homens
subject to Restricao_Homens:
    sum {v in Veiculos} h[v] = Homens_Total;

# Restrição de tempo para cada veículo
subject to Restricao_Tempo_Individuo {v in Veiculos}:
    Tempo_Viagem[v] * x[v] <= Tempo_Max;

# Limite de viagens para o veículo de passeio
subject to Limite_Veiculo_Passeio {v in Veiculos: v == 1}:
    x[v] == 1;  # Limita o veículo de passeio a apenas uma viagem

# Limite de viagens para outros veículos, podendo fazer múltiplas viagens
subject to Limite_Veiculos {v in Veiculos: v != 1}:
    x[v] <= Max_Viagens[v];


Overwriting evacuacao.mod


In [63]:
%%writefile evacuacao.dat
set Veiculos := 1 2 3 4 5 6;
set Familias := 1 2 3 4 5;

param Custo := 1 10 2 4 3 75 4 5 5 3 6 2;
param Capacidade_Pessoas := 1 20 2 5 3 10 4 30 5 15 6 5;
param Capacidade_Bagagem := 1 1000 2 20 3 50 4 1000 5 500 6 100;  # Multiplicado por 1000 (de toneladas para quilos)
param Tempo_Viagem := 1 60 2 45 3 10 4 45 5 30 6 30;
param Max_Viagens := 1 100 2 260 3 900 4 130 5 100 6 60;
param Tempo_Max := 600;

param Tamanho_Familia := 1 6 2 5 3 4 4 3 5 2;

param Quantidade_Familias :=
    1  10
    2  25
    3  150
    4  450
    5  350;

# Parâmetros de total de pessoas
param Homens_Total := 8000;  # Total de homens
param Mulheres_Total := 7900;  # Total de mulheres
param Criancas_Total := 1850;  # Total de crianças
param Pessoas_Total := 17750;
param Bagagem_Total := 177500;



Overwriting evacuacao.dat


In [64]:

%%ampl_eval
model evacuacao.mod;
data evacuacao.dat;
solve;
display x, y;

	presolve: constraint Transporte_Familias[5] cannot hold:
		body >= 350 cannot be <= 205; difference = 145
	presolve: constraint Transporte_Familias[4] cannot hold:
		body >= 450 cannot be <= 446; difference = 4
	presolve: constraint Transporte_Familias[3] cannot hold:
		body >= 150 cannot be <= 101; difference = 49
	presolve: constraint Restricao_Tempo cannot hold:
		body <= 540 cannot be >= 1245; difference = -705
x [*] :=
1  1
2  0
3  0
4  0
5  0
6  0
;

y [*,*] (tr)
:   1   2   3   4   5    :=
1   0   0   0   0   0
2   0   0   0   0   0
3   0   0   0   0   0
4   0   0   0   0   0
5   0   0   0   0   0
6   0   0   0   0   0
;



[AMPL Website](https://ampl.com) | [AMPL Colab](https://colab.ampl.com) | [Community Edition]( https://ampl.com/ce/) | [Twitter](https://twitter.com/AMPLopt) | [LinkedIn](https://www.linkedin.com/company/ampl)

[![Hits](https://h.ampl.com/https://github.com/ampl/amplcolab/blob/master/template/minimal.ipynb)](https://colab.ampl.com)